# Multi Greek Hedging

In [ ]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np

from utils.process import process_options,process_underlying,process_rf
from utils.greeks import delta, gamma, vega, theta, rho
from utils.implied_volatility import implied_vol_b76
from tqdm import tqdm

tqdm.pandas()


In [ ]:
option_data = pd.read_csv("../data/SPYOPT_processed.csv",parse_dates=True)
underlying = pd.read_csv("../data/SPYFUT_processed.csv",parse_dates=True).rename(columns={'close': 'futures_close'})
rf_rate = pd.read_csv("../data/RFRate.csv", parse_dates=True).rename(columns={'rate': 'rf_rate'})

In [ ]:
all_data = pd.merge(option_data, 
                    underlying[['ts_event', 'expiration_date', 'futures_close']], 
                    on=['ts_event', 'expiration_date'], 
                    how='left')

all_data = pd.merge(all_data, 
                    rf_rate[['ts_event', 'rf_rate']], 
                    on='ts_event', 
                    how='left')

all_data['rf_rate'] = all_data['rf_rate'].ffill().bfill()
all_data['rf_rate'] = 12 *np.log(1 + all_data['rf_rate']/1200)

all_data['ts_event'] = pd.to_datetime(all_data['ts_event'])
all_data['expiration_date'] = pd.to_datetime(all_data['expiration_date'])

all_data = all_data.set_index('ts_event')

all_data['futures_close'] = all_data['futures_close'].ffill()

all_data = all_data.reset_index()

all_data = all_data.drop(columns=['open', 'high', 'low', 'volume'])

all_data = all_data.rename(columns={'close': 'Value','futures_close': 'S','rf_rate': 'r','strike': 'K'})


all_data['T'] = (all_data['expiration_date'] - all_data['ts_event']).dt.days / 365.0

# Preparing Data

Since we are going to just be dealing with calls, we can filter out all the puts that are being traded.

In [ ]:
#Reindexing
def grouper1(group):
    start_date = group['ts_event'].min()
    end_date = group['ts_event'].max()
    group = group.set_index('ts_event').reindex(pd.date_range(start=start_date, end=end_date, freq='D')).reset_index()
    group['option_type'] = group['option_type'].ffill()
    group['K'] = group['K'].ffill()
    group['expiration_date'] = group['expiration_date'].ffill()
    return group

def filter1(group):
    if len(group) < 10:
        return False
    
    S0    = group['S'].iloc[0]                      # spot
    K     = group['K'].iloc[0]
    T    = group['T'].iloc[0]                      # time to maturity
    r = group['r'].iloc[0]                      # risk-free rate
    upper = S0 *np.exp(r*T + 1.64*0.2*np.sqrt(T))  # upper bound
    lower = S0 *np.exp(r*T - 1.64*0.2*np.sqrt(T))  # lower bound
    return (upper >= K >= lower)
    

all_data = all_data.groupby(['expiration_date','K','option_type']).filter(filter1).groupby(['expiration_date','K','option_type']).progress_apply(grouper1)

To accurately do the Greek hedging, you need to have the values for all greeks for each outstanding option at each time point.

Basically we need a big Tensor of shape  $ G \in\mathbb{R}^{n\times m\times d}$ with $n$ = timesteps (trading days), $m$ = outstanding options (strike prices and expiration dates), $d$ the greeks (delta,gamma,vega,theta,rho)

#### Small Cells

In [ ]:
all_data = all_data.reset_index(drop=True).rename(columns={'index': 'ts_event'})

In [ ]:
all_data['r'] = all_data['r'].ffill()
all_data['T'] = (all_data['expiration_date'] - all_data['ts_event']).dt.days / 365.0

In [ ]:
underlying['ts_event'] = pd.to_datetime(underlying['ts_event'])
underlying['expiration_date'] = pd.to_datetime(underlying['expiration_date'])

In [ ]:
all_data = pd.merge(all_data, 
                    underlying[['ts_event', 'expiration_date', 'futures_close']], 
                    on=['ts_event', 'expiration_date'], 
                    how='left')
all_data['S'] = all_data['S'].fillna(all_data['futures_close'])
all_data = all_data.drop(columns=['futures_close'])


In [ ]:
all_data = all_data[ all_data['ts_event'].dt.dayofweek < 5 ]

In [ ]:
def grouper2(group):
    group['S'] = group['S'].interpolate(method='spline', order=3)
    group['Value'] = group['Value'].interpolate(method='spline', order=3)
    return group

all_data = all_data.groupby(['expiration_date', 'K', 'option_type']).progress_apply(grouper2).reset_index(drop=True)


#### Splitting to Calls

In [ ]:
calls_data = all_data[all_data['option_type'] == 'C']

In [ ]:
calls_data['IV'] = calls_data.progress_apply(lambda row: implied_vol_b76(row['S'], row['K'], row['T'], row['r'], row['Value'], row['option_type']), axis=1)

In [ ]:
calls_data